# 합성곱 신경망 소개

이 노트북은 [케라스 창시자에게 배우는 딥러닝](https://tensorflow.blog/%EC%BC%80%EB%9D%BC%EC%8A%A4-%EB%94%A5%EB%9F%AC%EB%8B%9D/) 책의 5장 1절의 코드 예제를 참고했습니다.

----

다음 코드는 기본적인 컨브넷의 모습입니다. 
`Conv2D`와 `MaxPooling2D` 층을 쌓아 올렸습니다. 

컨브넷이 `(image_height, image_width, image_channels)` 크기의 입력 텐서를 사용한다는 점이 중요합니다(배치 차원은 포함하지 않습니다).

이 예제에서는 MNIST 이미지 포맷인 `(28, 28, 1)` 크기의 입력을 처리하도록 컨브넷을 설정해야 합니다. 이 때문에 첫 번째 층의 매개변수로 `input_shape=(28, 28, 1)`을 전달합니다.

### 코딩 시작 : 주석을 이용하여 None을 채우세요

In [20]:
from tensorflow.keras import layers
from tensorflow.keras import models

In [6]:
model = models.Sequential()

# 32개의 3x3 conv. filter, activation은 relu, input_shape은 (28, 28, 1)
model.add(layers.Conv2D(32, (3,3), activation='relu', input_shape=(28,28,1)))

In [8]:
# 2x2, stride 2의 maxpooling
model.add(layers.MaxPooling2D(pool_size=(2,2), strides=2))

In [9]:
# 64개의 3x3 conv. filter, activation은 relu
model.add(layers.Conv2D(64, (3,3), activation='relu'))

In [10]:
# 2x2, stride 2의 maxpooling
model.add(layers.MaxPooling2D(pool_size=(2,2), strides=2))

In [11]:
# 64개의 3x3 conv. filter, activation은 relu
model.add(layers.Conv2D(64, (3,3), activation='relu'))

지금까지 컨브넷 구조를 출력해 보죠:

In [12]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 3, 3, 64)          36928     
Total params: 55,744
Trainable params: 55,744
Non-trainable params: 0
_________________________________________________________________


`Conv2D`와 `MaxPooling2D` 층의 출력은 `(height, width, channels)` 크기의 3D 텐서입니다. 높이와 넓이 차원은 네트워크가 깊어질수록 작아지는 경향이 있습니다. 채널의 수는 `Conv2D` 층에 전달된 첫 번째 매개변수에 의해 조절됩니다(32개 또는 64개).

다음 단계에서 마지막 층의 (`(3, 3, 64)` 크기인) 출력 텐서를 완전 연결 네트워크에 주입합니다. 이 네트워크는 이미 익숙하게 보았던 `Dense` 층을 쌓은 분류기입니다. 이 분류기는 1D 벡터를 처리하는데 이전 층의 출력이 3D 텐서입니다. 그래서 먼저 3D 출력을 1D 텐서로 펼쳐야 합니다. 그다음 몇 개의 `Dense` 층을 추가합니다:

### 코딩 시작 : 주석을 이용하여 None을 채우세요

In [13]:
#Layers.Flatten() -> X x Y x Z image를 한줄로 쫙 핌
model.add(layers.Flatten())

# 64개의 Unit추가, activation은 relu
model.add(layers.Dense(64, activation='relu'))

# 10개 (클래스의 수)의 Unit추가, activation은 softmax
model.add(layers.Dense(10, activation='softmax'))

10개의 클래스를 분류하기 위해 마지막 층의 출력 크기를 10으로 하고 소프트맥스 활성화 함수를 사용합니다. 이제 전체 네트워크는 다음과 같습니다:

In [14]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 3, 3, 64)          36928     
_________________________________________________________________
flatten (Flatten)            (None, 576)               0         
_________________________________________________________________
dense (Dense)                (None, 64)               

여기에서 볼 수 있듯이 `(3, 3, 64)` 출력이 `(576,)` 크기의 벡터로 펼쳐진 후 `Dense` 층으로 주입되었습니다.

이제 MNIST 숫자 이미지에 이 컨브넷을 훈련합니다. 

In [15]:
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical

(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

train_images = train_images.reshape((60000, 28, 28, 1))
train_images = train_images.astype('float32') / 255

test_images = test_images.reshape((10000, 28, 28, 1))
test_images = test_images.astype('float32') / 255

train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)

In [16]:
import matplotlib.pyplot as plt
train_images[0].shape
plt.imshow(train_images[0].reshape(28,28))
print(train_labels[0])

[0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]


In [17]:
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
model.fit(train_images, train_labels, epochs=5, batch_size=64)

Train on 60000 samples
Epoch 1/5
60000/60000 [==============================] - 22s 363us/sample - loss: 0.1694 - accuracy: 0.9467
Epoch 2/5
60000/60000 [==============================] - 21s 349us/sample - loss: 0.0475 - accuracy: 0.9852
Epoch 3/5
60000/60000 [==============================] - 22s 361us/sample - loss: 0.0325 - accuracy: 0.9901
Epoch 4/5
60000/60000 [==============================] - 22s 371us/sample - loss: 0.0243 - accuracy: 0.9925
Epoch 5/5
60000/60000 [==============================] - 22s 371us/sample - loss: 0.0182 - accuracy: 0.9942


테스트 데이터에서 모델을 평가해 보죠:

In [18]:
test_loss, test_acc = model.evaluate(test_images, test_labels)

10000/1 [===============================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================

In [19]:
test_acc

0.9888

2장의 완전 연결 네트워크는 97.8%의 테스트 정확도를 얻은 반면, 기본적인 컨브넷은 99.2%의 테스트 정확도를 얻었습니다.